# agentscope.agent

agentscope.agent 模块是 AgentScope  框架中用于构建智能体（Agent）的核心模块。它提供了一套灵活、可扩展的异步智能体基类和常用实现（如 ReActAgent、UserAgent），并支持钩子（hook）、消息观察（observe）、结构化输出、工具调用、记忆管理等高级功能。

一、核心类结构 
1. AgentBase 

    所有智能体的基类，继承自 StateModule。
    支持异步操作（async/await）。
    提供消息处理、观察、打印、中断处理等基础能力。
    支持 实例级 和 类级 的钩子系统（hook system）。
     

支持的 hook 类型：
['pre_reply', 'post_reply', 'pre_print', 'post_print', 'pre_observe', 'post_observe']
 
2. ReActAgentBase 与 ReActAgent 

    实现了 ReAct（Reason + Act） 算法框架。
    在 AgentBase 基础上增加了：
        reasoning() 和 acting() 抽象接口（由元类自动调用）
        新增 hook 类型：pre_reasoning, post_reasoning, pre_acting, post_acting
         
    支持：
        工具调用（串行/并行）
        长期记忆（Long-term Memory）
        知识库检索（Knowledge Retrieval）
        结构化输出（Structured Output）
        实时干预（Realtime Steering）
         

3. UserAgent 

    用于模拟“用户”角色，接收真实用户输入（如终端、Web UI）。
    可通过 UserInputBase 子类（如 TerminalUserInput, StudioUserInput）自定义输入源。
     

In [3]:
from agentscope.agent import ReActAgent
from agentscope.model import DashScopeChatModel
from agentscope.message import Msg
from agentscope.formatter import DashScopeChatFormatter

import os
from dotenv import load_dotenv
load_dotenv()
# 初始化模型（需配置 API key）
model = DashScopeChatModel(
            model_name="qwen-max",
            api_key=os.environ["DASHSCOPE_API_KEY"],
            stream=False,
        )

# 使用默认 formatter
formatter = DashScopeChatFormatter()

# 创建 ReAct 智能体
agent = ReActAgent(
    name="Assistant",
    sys_prompt="你是一个乐于助人的 AI 助手。",
    model=model,
    formatter=formatter,
    max_iters=5,
)

# 发送消息并获取回复
user_msg = Msg(name="User", content="天空为什么是蓝色", role="user")
response = await agent.reply(user_msg)
print(response.content)

Assistant: 天空之所以呈现蓝色，主要是因为大气中的分子和微小颗粒对太阳光的散射作用。这种现象被称为瑞利散射（Rayleigh scattering）。当阳光进入地球大气层时，它包含了多种颜色的光，这些颜色组合在一起看起来是白色的。但是，不同颜色的光其波长不同，蓝光的波长较短，在遇到气体分子等细小物体时更容易被散射到各个方向。因此当我们抬头看天空时，看到的是被散射了的蓝光居多，使得天空呈现出蔚蓝色。值得注意的是，在日出和日落时分，由于光线需要穿过更厚的大气层，此时蓝光几乎完全被散射掉了，而波长较长的红光则更为显著，所以这时我们能看到更加绚丽多彩的晚霞或朝霞景色。
天空之所以呈现蓝色，主要是因为大气中的分子和微小颗粒对太阳光的散射作用。这种现象被称为瑞利散射（Rayleigh scattering）。当阳光进入地球大气层时，它包含了多种颜色的光，这些颜色组合在一起看起来是白色的。但是，不同颜色的光其波长不同，蓝光的波长较短，在遇到气体分子等细小物体时更容易被散射到各个方向。因此当我们抬头看天空时，看到的是被散射了的蓝光居多，使得天空呈现出蔚蓝色。值得注意的是，在日出和日落时分，由于光线需要穿过更厚的大气层，此时蓝光几乎完全被散射掉了，而波长较长的红光则更为显著，所以这时我们能看到更加绚丽多彩的晚霞或朝霞景色。


In [6]:
def log_pre_reply(agent, *args, **kwargs):
    print(f"[Hook] {agent.name} 即将开始回复...")

def log_post_reply(agent, result, *args, **kwargs):
    # 处理不同的结果格式
    if hasattr(result, 'content'):
        # 如果是 Msg 对象
        content = result.content
    elif isinstance(result, dict):
        # 如果是字典，尝试获取 content 或其他字段
        content = result.get('content', result.get('text', str(result)))
    else:
        # 其他情况，直接转为字符串
        content = str(result)
    
    print(f"[Hook] {agent.name} 回复完成: {content}")

# 注册实例级钩子
agent.register_instance_hook("pre_reply", "log_start", log_pre_reply)
agent.register_instance_hook("post_reply", "log_end", log_post_reply)

# 调用 reply 会触发钩子
await agent.reply(Msg("User", "你好！", "user"))

[Hook] Assistant 即将开始回复...
Assistant: 你好！有什么我可以帮到你的吗？
[Hook] Assistant 回复完成: {'msg': Msg(id='h8eZjxqBevEAm4QvNjvDG9', name='User', content='你好！', role='user', metadata=None, timestamp='2025-10-02 23:25:06.639', invocation_id='None'), 'structured_model': None}


Msg(id='Si5Zv7iepasAnsgzNDAK37', name='Assistant', content='你好！有什么我可以帮到你的吗？', role='assistant', metadata=None, timestamp='2025-10-02 23:25:07.690', invocation_id='None')

In [7]:
from agentscope.agent import UserAgent, TerminalUserInput

# 创建用户代理，从终端读取输入
user_agent = UserAgent(name="Human")
user_agent.override_instance_input_method(TerminalUserInput(input_hint="请输入你的问题: "))

# 等待用户输入
user_msg = await user_agent.reply()
print(f"用户输入: {user_msg.content}")

Human: 你好
用户输入: 你好
